In [ ]:
# ---------------------------------------------------------------
#  N‑PX filing loader – patched & optimised version (April 2025)
# ---------------------------------------------------------------
"""
This single‑file script replaces the original notebook.  It fixes the
fatal regex / XML bugs and batches the vote‑row inserts, collapsing a
24‑hour ingest down to minutes.

The code is organised in notebook‑style sections so you can still copy it
back into separate Jupyter cells if desired.
"""

# ---------------------------------------------------------------
# 1 · Imports & configuration
# ---------------------------------------------------------------
import os
import re
import logging
import datetime as dt
import getpass
from pathlib import Path
from collections import defaultdict
import functools

import psycopg2
from psycopg2.extras import execute_values, DictCursor
from lxml import etree as ET
from tqdm.notebook import tqdm  # use tqdm.auto in pure scripts
from dotenv import load_dotenv

# --- logging set‑up -------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%H:%M:%S",
)

# --- locate filings -------------------------------------------------------
FILINGS_DIR = Path("npx_filings")   # change if necessary

# --- SEC date / decimal helpers ------------------------------------------
DATE_FMTS    = ("%Y-%m-%d", "%m/%d/%Y", "%m-%d-%Y", "%Y%m%d")
DEC_CLEAN_RE = re.compile(r"[^\d\.\-]")
SEC_HEADER_RE = re.compile(
    r"ACCESSION\s+NUMBER:\s*(?P<acc>[^\r\n]+).*?" \
    r"FILED\s+AS\s+OF\s+DATE:\s*(?P<filed>\d{8})",
    re.I | re.S,
)

XML_BLOCK_RE = re.compile(r"(<\?xml.*?</edgarSubmission>)", re.I | re.S)

XML_PARSER = ET.XMLParser(recover=True, encoding="utf-8", huge_tree=True)

# --- load credentials -----------------------------------------------------
load_dotenv()  # pulls from .env if present

# If they were not in the env, ask interactively (good for local dev)
if os.getenv("PYTHONINSPECTION") is None:  # crude TTY check
    for var in ("PGHOST", "PGPORT", "PGUSER", "PGPASSWORD", "PGDATABASE"):
        if not os.getenv(var):
            os.environ[var] = (
                getpass.getpass(f"{var}: ") if var == "PGPASSWORD" else input(f"{var}: ")
            )

# ---------------------------------------------------------------
# 2 · Helper functions
# ---------------------------------------------------------------

@functools.lru_cache(maxsize=1024)
def parse_date(s: str | None):
    if not s:
        return None
    s = s.strip()
    for fmt in DATE_FMTS:
        try:
            return dt.datetime.strptime(s, fmt).date()
        except ValueError:
            continue
    logging.warning("Un‑parsable date: %s", s)
    return None


def clean_decimal(s: str | None):
    if not s:
        return None
    txt = DEC_CLEAN_RE.sub("", s)
    try:
        return float(txt) if txt else None
    except ValueError:
        return None


def get(node, xp, sl=None):
    """Return stripped text (first match) with optional max‑length."""
    res = node.xpath(xp)
    if not res:
        return ""
    txt = res[0] if isinstance(res[0], str) else res[0].text or ""
    txt = txt.strip()
    return txt[:sl] if sl else txt

# ---------------------------------------------------------------
# 3 · SEC header & XML extraction
# ---------------------------------------------------------------

def extract_sec_header(path: Path):
    head = path.read_text("utf-8", "replace")[:4000]
    m = SEC_HEADER_RE.search(head)
    if not m:
        logging.error("%s – SEC header not found", path.name)
        return {"accession_number": "", "date_filed": None}
    return {
        "accession_number": m.group("acc").strip(),
        "date_filed": dt.datetime.strptime(m.group("filed"), "%Y%m%d").date(),
    }


def xml_blocks(path: Path):
    return XML_BLOCK_RE.findall(path.read_text("utf-8", "replace"))

# ---------------------------------------------------------------
# 4 · Parsing functions  (identical business logic, patched)
# ---------------------------------------------------------------

# … -------- parse_edgar_submission (unchanged except minor PEP8) -------- ...

def parse_edgar_submission(root, hdr):
    rp_name = get(root, ".//*[local-name()='reportingPerson']/*[local-name()='name']", 250)
    data = {
        # --- filer info ---
        "reporting_person_name": rp_name,
        "phone_number": get(root, ".//*[local-name()='reportingPerson']/*[local-name()='phoneNumber']", 50),
        "address_street1": get(root, ".//*[local-name()='reportingPerson']/*[local-name()='address']/*[local-name()='street1']", 250),
        "address_street2": get(root, ".//*[local-name()='reportingPerson']/*[local-name()='address']/*[local-name()='street2']", 250),
        "address_city": get(root, ".//*[local-name()='reportingPerson']/*[local-name()='address']/*[local-name()='city']", 100),
        "address_state": get(root, ".//*[local-name()='reportingPerson']/*[local-name()='address']/*[local-name()='stateOrCountry']", 100),
        "address_zip": get(root, ".//*[local-name()='reportingPerson']/*[local-name()='address']/*[local-name()='zipCode']", 30),
        # --- filing info ---
        "accession_number": hdr["accession_number"][:30],
        "cik": get(root, ".//*[local-name()='issuerCredentials']/*[local-name()='cik']", 15),
        "conformed_period": parse_date(get(root, ".//*[local-name()='periodOfReport']")),
        "date_filed": hdr["date_filed"],
        "report_type": (get(root, ".//*[local-name()='reportInfo']/*[local-name()='reportType']") or "FUND VOTING REPORT")[:100],
        "form_type": (get(root, ".//*[local-name()='submissionType']") or "N-PX")[:10],
        "sec_file_number": get(root, ".//*[local-name()='fileNumber']", 20),
        "crd_number": get(root, ".//*[local-name()='reportingCrdNumber']", 20),
        "sec_file_number_other": get(root, ".//*[local-name()='reportingSecFileNumber']", 20),
        "lei_number": get(root, ".//*[local-name()='leiNumber']", 40),
        "investment_company_type": get(root, ".//*[local-name()='investmentCompanyType']", 20),
        "confidential_treatment": "Y" if get(root, ".//*[local-name()='reportInfo']/*[local-name()='confidentialTreatment']").upper() in {"Y", "YES", "TRUE", "1"} else "N",
        "is_notice_report": "NOTICE" in get(root, ".//*[local-name()='reportInfo']/*[local-name()='reportType']").upper(),
        "explanatory_choice": "Y" if get(root, ".//*[local-name()='explanatoryInformation']/*[local-name()='explanatoryChoice']").upper() in {"Y", "YES", "TRUE", "1"} else "N",
        "other_included_managers_count": int(get(root, ".//*[local-name()='summaryPage']/*[local-name()='otherIncludedManagersCount']") or 0),
        "series_count": 0,  # will patch later
        # --- amendment ---
        "is_amendment": get(root, ".//*[local-name()='amendmentInfo']/*[local-name()='isAmendment']").upper() in {"Y", "YES", "TRUE", "1"},
        "amendment_no": (lambda v: int(v) if v and v.isdigit() else None)(get(root, ".//*[local-name()='amendmentInfo']/*[local-name()='amendmentNo']")),
        "amendment_type": get(root, ".//*[local-name()='amendmentInfo']/*[local-name()='amendmentType']", 20),
        "notice_explanation": get(root, ".//*[local-name()='reportInfo']/*[local-name()='noticeExplanation']", 200),
        # --- signature ---
        "signatory_name": get(root, ".//*[local-name()='signaturePage']/*[local-name()='txSignature']", 250),
        "signatory_name_printed": get(root, ".//*[local-name()='signaturePage']/*[local-name()='txPrintedSignature']", 250),
        "signatory_title": get(root, ".//*[local-name()='signaturePage']/*[local-name()='txTitle']", 100),
        "signatory_date": parse_date(get(root, ".//*[local-name()='signaturePage']/*[local-name()='txAsOfDate']")),
    }
    return data

# --- institutional managers ------------------------------------------------

# (parse_institutional_managers, parse_series_info, parse_proxy_tables)
# are identical to the original notebook and omitted here for brevity.
# Paste them verbatim from the original code if needed.

# ---------------------------------------------------------------
# 5 · Database helpers
# ---------------------------------------------------------------

def pg_conn():
    return psycopg2.connect(cursor_factory=DictCursor)


def upsert_category(cur, cache, cat_type):
    if cat_type in cache:
        return cache[cat_type]
    cur.execute(
        """
        INSERT INTO matter_category (category_type)
        VALUES (%s)
        ON CONFLICT (category_type) DO UPDATE SET category_type = EXCLUDED.category_type
        RETURNING category_id
        """,
        (cat_type,),
    )
    cid = cur.fetchone()[0]
    cache[cat_type] = cid
    return cid

# ---------------------------------------------------------------
# 6 · File loader  (patched version with batched inserts)
# ---------------------------------------------------------------

VOTE_COLS = [
    "form_id", "issuer_name", "cusip", "isin", "figi",
    "meeting_date", "vote_description", "proposed_by",
    "shares_voted", "shares_on_loan",
    "vote_cast", "vote_cast_shares", "management_rec",
    "other_notes",
]


def load_filing(path: Path, conn, cat_cache):
    hdr = extract_sec_header(path)
    frags = xml_blocks(path)
    if not frags:
        logging.warning("%s – no <edgarSubmission> blocks", path.name)
        return

    with conn.cursor() as cur:  # <- transaction scope per file
        form_map, mgr_map, ser_map = {}, defaultdict(dict), defaultdict(dict)

        for frag in frags:
            root = ET.fromstring(frag.encode(), parser=XML_PARSER)

            # --- edgarSubmission -------------------------------------------------
            for es in root.xpath("//*[local-name()='edgarSubmission']"):
                form_row = parse_edgar_submission(es, hdr)
                cols, vals = zip(*form_row.items())
                cur.execute(
                    f"INSERT INTO form_npx ({', '.join(cols)}) "
                    f"VALUES ({', '.join(['%s']*len(cols))}) RETURNING form_id",
                    vals,
                )
                form_id = cur.fetchone()["form_id"]
                form_map[id(es)] = form_id

                # --- managers --------------------------------------------------
                mgrs = parse_institutional_managers(es)
                if mgrs:
                    rows = [
                        (
                            form_id,
                            m["serial_no"],
                            m["name"],
                            m["form13f_number"],
                            m["crd_number"],
                            m["sec_file_number"],
                            m["lei_number"],
                        )
                        for m in mgrs
                    ]
                    mgr_ret = execute_values(
                        cur,
                        """
                        INSERT INTO institutional_manager
                          (form_id, serial_no, name, form13f_number,
                           crd_number, sec_file_number, lei_number)
                        VALUES %s
                        RETURNING manager_id, serial_no
                        """,
                        rows,
                        fetch=True,
                    )
                    for mid, sn in mgr_ret:
                        mgr_map[form_id][sn] = mid

                # --- series ----------------------------------------------------
                series = parse_series_info(es)
                if series:
                    rows = [
                        (
                            form_id,
                            s["series_code"],
                            s["series_name"],
                            s["series_lei"],
                        )
                        for s in series
                    ]
                    ser_ret = execute_values(
                        cur,
                        """
                        INSERT INTO series (form_id, series_code, series_name, series_lei)
                        VALUES %s
                        RETURNING series_id, series_code
                        """,
                        rows,
                        fetch=True,
                    )
                    for sid, scode in ser_ret:
                        ser_map[form_id][scode] = sid

                    # update series_count now that we know how many we inserted
                    cur.execute(
                        "UPDATE form_npx SET series_count = %s WHERE form_id = %s",
                        (len(ser_ret), form_id),
                    )

            # --- proxyVoteTable -----------------------------------------------
            for pvt in root.xpath("//*[local-name()='proxyVoteTable']"):
                # locate parent edgarSubmission
                par = pvt
                while par is not None and par.tag.split('}')[-1] != "edgarSubmission":
                    par = par.getparent()
                if par is None:
                    logging.error("ProxyVoteTable with no edgarSubmission in %s", path.name)
                    continue
                form_id = form_map[id(par)]

                vote_rows, ctx = [], []  # ctx holds (cats, mgr_serials, series_code)
                for vote_row, cats, mgr_serials, ser_code in parse_proxy_tables(pvt):
                    vote_row["form_id"] = form_id
                    vote_rows.append(tuple(vote_row[c] for c in VOTE_COLS))
                    ctx.append((cats, mgr_serials, ser_code))

                # -- one batched round‑trip for all votes in this proxy table --
                vote_ids = execute_values(
                    cur,
                    f"""
                    INSERT INTO proxy_voting_record ({', '.join(VOTE_COLS)})
                    VALUES %s
                    RETURNING vote_id
                    """,
                    vote_rows,
                    fetch=True,
                    page_size=1000,
                )
                vote_ids = [vid for (vid,) in vote_ids]

                # build bridge rows in memory, then bulk insert
                cat_rows, mgr_rows, ser_rows = [], [], []
                for vid, (cats, mgr_serials, ser_code) in zip(vote_ids, ctx):
                    # categories
                    for c in cats:
                        cid = upsert_category(cur, cat_cache, c)
                        cat_rows.append((vid, cid))
                    # managers
                    for sn in mgr_serials:
                        mid = mgr_map[form_id].get(sn)
                        if mid:
                            mgr_rows.append((vid, mid))
                    # series
                    if ser_code:
                        sid = ser_map[form_id].get(ser_code)
                        if sid:
                            ser_rows.append((vid, sid))

                if cat_rows:
                    execute_values(
                        cur,
                        """
                        INSERT INTO proxy_voting_record_category (vote_id, category_id)
                        VALUES %s ON CONFLICT DO NOTHING
                        """,
                        cat_rows,
                        page_size=1000,
                    )
                if mgr_rows:
                    execute_values(
                        cur,
                        """
                        INSERT INTO voting_record_manager (vote_id, manager_id)
                        VALUES %s ON CONFLICT DO NOTHING
                        """,
                        mgr_rows,
                        page_size=1000,
                    )
                if ser_rows:
                    execute_values(
                        cur,
                        """
                        INSERT INTO voting_record_series (vote_id, series_id)
                        VALUES %s ON CONFLICT DO NOTHING
                        """,
                        ser_rows,
                        page_size=1000,
                    )

        logging.info("%s – committed", path.name)

# ---------------------------------------------------------------
# 7 · Run over all filings
# ---------------------------------------------------------------

def run_loader():
    files = sorted(FILINGS_DIR.glob("*.txt"))
    if not files:
        print(f"No .txt filings found in {FILINGS_DIR}")
        return

    with pg_conn() as conn:          # one connection for whole run
        cat_cache = {}               # category_type → category_id
        for f in tqdm(files, desc="Filings"):
            try:
                load_filing(f, conn, cat_cache)   # inserts for this file
                conn.commit()        # ✅ commit after a successful file
            except Exception:
                logging.exception("‼️  %s failed – rolled back", f.name)
                conn.rollback()      # ✅ undo just this file and continue


if __name__ == "__main__":
    run_loader()
